In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup


import datetime
import time
import sys
import os


sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
browser =  webdriver.Chrome()
browser.get("https://jobs.dou.ua/")

require_title = ["Python"]

In [ ]:
def vacancies_list_processing(el):
    pars_vacancies = dict()

    el.click()

    vacancies = browser.find_elements(By.CLASS_NAME, 'l-vacancy')

    for vac in vacancies:
        
        vac.find_element(By.TAG_NAME, 'a').click()

        pars_vacancies[browser.current_url] = browser.page_source

        time.sleep(1)

        browser.back()
    
    return pars_vacancies

In [4]:
pars_vacancies = None

categories = browser.find_elements(By.CLASS_NAME, 'cat-link')

for cat in categories:
    try:
        if (cat.text in require_title):
            pars_vacancies = vacancies_list_processing(cat)
    except:
        pass

browser.close()

In [ ]:
data = list()

replace_month_dict = {
    'січня':      1,
    'лютого':     2,
    'березня':    3,
    'травня':     4,
    'квітня':     5,
    'червня':     6,
    'липня':      7,
    'серпня':     8,
    'вересня':    9,
    'жовтня':     10,
    'листопада':  11,
    'грудня':     12,
}

for i in pars_vacancies.items():
    vacansy = dict()
    

    res = BeautifulSoup(i[1], 'lxml')
    res = res.find_all("div", class_="l-vacancy")[0]

    vacansy['link'] = i[0]
    vacansy['data'] = i[1]
    vacansy['name'] = res.find("h1").text
    date = str(res.find("div", class_='date').text).replace('\n', '').replace('\t', '').split(" ")
    vacansy['published_at'] = datetime.datetime(int(date[2]), replace_month_dict[date[1].lower()], int(date[0]))
    
    data.append(vacansy)

In [ ]:
from src.database import async_session
from src.models.vacancies import RawVacancies

In [ ]:
async with async_session() as session:
    for d in data:
        rvac = RawVacancies(**d)
        rvac.source_id = 1
        rvac.created_at = datetime.datetime.now()
        rvac.updated_at = datetime.datetime.now()
        session.add(rvac)
        await session.commit()

2025-08-08 19:32:16,553 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-08 19:32:16,555 INFO sqlalchemy.engine.Engine INSERT INTO raw_vacancies (source_id, link, name, data, published_at, unpublished_at, created_at, updated_at) VALUES ($1::INTEGER, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR, $5::TIMESTAMP WITHOUT TIME ZONE, $6::TIMESTAMP WITHOUT TIME ZONE, $7::TIMESTAMP WITHOUT TIME ZONE, $8::TIMESTAMP WITHOUT TIME ZONE) RETURNING raw_vacancies.raw_vacancy_id
2025-08-08 19:32:16,556 INFO sqlalchemy.engine.Engine [cached since 89.2s ago] (1, 'https://jobs.dou.ua/companies/verybusy-recruiting/vacancies/317934/?from=list_hot', 'Senior Backend (Python) Engineer', '<html lang="uk" xmlns:og="http://ogp.me/ns#" class="is_desktop" data-theme="auto"><head>\n\t<meta charset="utf-8">\n\n\t<title>Senior Backend (Python ... (45815 characters truncated) ... ;uam=&amp;uap=Windows&amp;uapv=19.0.0&amp;uaw=0&amp;fledge=1&amp;_tu=Cg" style="display: none; visibility: hidden;"></iframe>\n\n\n\n\n</body>